In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path_train = '1000_train.csv'
train_data=pd.read_csv(path_train)
samples = train_data.sample(n=700000)

In [3]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
classes = 2
Y = to_categorical(Y,classes)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(83886, 27) (83886, 2)
(20972, 27) (20972, 2)


In [5]:
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)
y_train = np.argmax(Y_train,axis=-1)
y_test = np.argmax(Y_test,axis=-1)
print(y_train.shape)

(83886,)


In [6]:
from tab2img.converter import Tab2Img
model = Tab2Img()
train = model.fit_transform(X_train, y_train)
test = model.transform(X_test)
print(train.shape,test.shape)

(83886, 6, 6) (20972, 6, 6)


In [7]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(train)):
    img = Image.fromarray(train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(test)):
    img = Image.fromarray(test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(83886, 32, 32, 3) (20972, 32, 32, 3)


In [8]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=20,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/20
328/328 [==============================] - 30s 93ms/step - loss: 0.2533 - accuracy: 0.9028 - val_loss: 1.7674 - val_accuracy: 0.5283
Epoch 2/20
328/328 [==============================] - 23s 69ms/step - loss: 0.2299 - accuracy: 0.9110 - val_loss: 0.2915 - val_accuracy: 0.9032
Epoch 3/20
328/328 [==============================] - 23s 70ms/step - loss: 0.2282 - accuracy: 0.9123 - val_loss: 0.2265 - val_accuracy: 0.9130
Epoch 4/20
328/328 [==============================] - 23s 70ms/step - loss: 0.2257 - accuracy: 0.9125 - val_loss: 0.2490 - val_accuracy: 0.9074
Epoch 5/20
328/328 [==============================] - 23s 70ms/step - loss: 0.2223 - accuracy: 0.9135 - val_loss: 0.2214 - val_accuracy: 0.9123
Epoch 6/20
328/328 [==============================] - 23s 71ms/step - loss: 0.2227 - accuracy: 0.9145 - val_loss: 0.2347 - val_accuracy: 0.9052
Epoch 7/20
328/328 [==============================] - 23s 71ms/step - loss: 0.2399 - accuracy: 0.9087 - val_loss: 0.2334 - val_accuracy:

In [9]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

2622/2622 [==============================] - 27s 10ms/step - loss: 0.2156 - accuracy: 0.9156
training accuracy:  0.915587842464447
testing accuracy:  0.9168415069580078
